In [1]:
include("../../../main.jl")
using PyCall;
using Revise;
using Gen;
using CairoMakie
using Serialization
sepsis_gym = pyimport("custom_sepsis");

In [3]:
histories = Dict(
    :Softmax => [load_jld("../../../data/mcmc/runs/history-SoftmaxPPL-$i.jld")  for i in 1:10],
    :SimplePPL => [load_jld("../../../data/mcmc/runs/history-SimplePPL-$i.jld")  for i in 1:10]
)

Dict{Symbol, Vector{RunData}} with 2 entries:
  :Softmax   => [RunData("history-SoftmaxPPL-1", :Softmax, ModelData(DynamicCho…
  :SimplePPL => [RunData("history-SimplePPL-1", :SimplePPL, ModelData(DynamicCh…

In [2]:
# histories = Dict(
#     :SimplePPL => [RunData("history-SimplePPL-$(i)", :SimplePPL) for i in 1:10],
#     :Softmax => [RunData("history-SoftmaxPPL-$(i)", :Softmax) for i in 1:10],
# )

Dict{Symbol, Vector{RunData}} with 2 entries:
  :Softmax   => [RunData("history-SoftmaxPPL-1", :Softmax, ModelData(DynamicCho…
  :SimplePPL => [RunData("history-SimplePPL-1", :SimplePPL, ModelData(DynamicCh…

In [3]:

indices = [2^i for i in [0,1,2,3,4,5,6,8,10,12,14,17,20,26,]]

batch_sizes = [indices[i] - indices[i-1] for i in 2:length(indices)]


13-element Vector{Int64}:
        1
        2
        4
        8
       16
       32
      192
      768
     3072
    12288
   114688
   917504
 66060288

In [1]:
batch_sizes = filter(x -> x > 192, batch_sizes)

LoadError: UndefVarError: `batch_sizes` not defined

In [ ]:
for batch_size in batch_sizes
    for i in 1:10
        for type in [:SimplePPL, :Softmax]
            histories[type][i] = history_run(histories[type][i], batch_size, 1)
            println("$(type) $(i) $(batch_size)")
        end
    end
end

In [ ]:

# struct HistoryType
#     mean_rewards::Dict{Int,Vector{Float64}}
#     name::String
#     info::Dict
# end


In [ ]:
# RunData
# name::String
# type::Symbol
# model::ModelData
# mcmcs::Dict{Int,MCMCRun}
# mean_rewards::Dict{Int,Float64}
# index::Int

In [12]:

function evaluate_on_samples_local(mcmc::MCMCRun, functions, n_samples::Int)
    # perform planning
    posterior_size = Int(round(length(mcmc.scores[end]) / 3))
    posterior = mcmc.params[end-posterior_size:end]
    mean_rewards = []
    sampled_params = []
    policies = []
    for i in 1:n_samples
        param = rand(posterior)
        push!(sampled_params, param)
        policy, V = optimize(param, functions)
        push!(policies, policy)
        # evaluate policy
        mean_reward = sepsis_gym.evaluate_policy(to_gym_pol(policy), 100000)
        push!(mean_rewards, mean_reward)
    end

    return mean_rewards, policies, sampled_params
end

evaluate_on_samples_local (generic function with 1 method)

In [13]:
history_types = []
for run in histories[:SimplePPL]
    mean_rewards = Dict()
    for (k, mcmc) in run.mcmcs
        mean_rewards[k] = evaluate_on_samples_local(mcmc, simple_functions, 10)
        println("$(run.name) $(k) $(mean_rewards[k])")
    end
    push!(history_types, HistoryType(mean_rewards, "SimplePPL", Dict("description" => "At every checkpoint, 10 samples are drawn from the posterior")))
end


LoadError: MethodError: [0mCannot `convert` an object of type [92mTuple{Vector{Any}, Vector{Any}, Vector{Any}}[39m[0m to an object of type [91mVector{Float64}[39m

[0mClosest candidates are:
[0m  convert(::Type{Array{T, N}}, [91m::StaticArraysCore.SizedArray{S, T, N, N, Array{T, N}}[39m) where {S, T, N}
[0m[90m   @[39m [33mStaticArrays[39m [90m~/.julia/packages/StaticArrays/xEhFV/src/[39m[90m[4mSizedArray.jl:88[24m[39m
[0m  convert(::Type{Array{T, N}}, [91m::StaticArraysCore.SizedArray{S, T, N, M, TData} where {M, TData<:AbstractArray{T, M}}[39m) where {T, S, N}
[0m[90m   @[39m [33mStaticArrays[39m [90m~/.julia/packages/StaticArrays/xEhFV/src/[39m[90m[4mSizedArray.jl:82[24m[39m
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:84[24m[39m
[0m  ...
